<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">


# Predicting Shots Made Per Game by Kobe Bryant

_Authors: Kiefer Katovich (SF)_

---

In this lab you'll be using regularized regression penalties — ridge, lasso, and elastic net — to try and predict how many shots Kobe Bryant made per game during his career.

The Kobe Shots data set contains hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model, considering the limited number of observations (games) we have available. Plus, many of the predictors have significant multicollinearity. 


**Warning:** Some of these calculations are computationally expensive and may take a while to execute. It may be worthwhile to only use a portion of the data to perform these calculations, especially if you've experienced kernel issues in the past.

---

### 1) Load packages and data.

In [34]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [35]:
kobe = pd.read_csv('./datasets/kobe_superwide_games.csv')

---

### 2) Examine the data.

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for modeling this data?

In [36]:
kobe.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [37]:
kobe.columns

Index(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05',
       ...
       'ACTION_TYPE:tip_layup_shot', 'ACTION_TYPE:tip_shot',
       'ACTION_TYPE:turnaround_bank_shot',
       'ACTION_TYPE:turnaround_fadeaway_bank_jump_shot',
       'ACTION_TYPE:turnaround_fadeaway_shot',
       'ACTION_TYPE:turnaround_finger_roll_shot',
       'ACTION_TYPE:turnaround_hook_shot', 'ACTION_TYPE:turnaround_jump_shot',
       'SEASON_GAME_NUMBER', 'CAREER_GAME_NUMBER'],
      dtype='object', length=645)

---

### 3) Create predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

Predictor(X) and Target(y)

In [38]:
X= kobe.drop('SHOTS_MADE', axis=1)
y=kobe['SHOTS_MADE']

Scale the Data

In [39]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

In [40]:
X_Scaled = ss.fit_transform(X)

---

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [41]:
model = LinearRegression()

In [51]:
print(cross_val_score(model, X_Scaled,y,cv=10))
print(np.mean(cross_val_score(model, X_Scaled,y,cv=10)))

[-8.21974827e+28 -8.93366955e+27 -1.72949798e+27 -2.55961745e+27
 -4.41730145e+27 -6.82076677e+26 -3.23598335e+27 -1.69338500e+27
 -1.82179630e+27 -8.03804534e+27]
-1.153088558418262e+28


Negative $R^2$ scores mean the models are doing very badly

---

### 5) Find an optimal value for the ridge regression alpha using `RidgeCV`.

Go to the documentation and [read how RidgeCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).

> *Hint: Once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that ridge performs best when searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit.


**Refer to Lecture 3.06**

In [52]:
from sklearn.linear_model import Ridge

In [53]:
ridge_alphas = np.logspace(0, 5, 200)

Source for Optimal Ridge Alpha 

https://chrisalbon.com/code/machine_learning/linear_regression/selecting_best_alpha_value_in_ridge_regression/

Logspace

https://www.geeksforgeeks.org/numpy-logspace-python/

In [54]:
best_ridge = RidgeCV(alphas=ridge_alphas, cv=10)

In [55]:
best_ridge = best_ridge.fit(X_Scaled,y)

In [56]:
print(best_ridge.alpha_)

775.2597488629465


Best Alpha value is 775.25

---

### 6) Cross-validate the ridge regression $R^2$ with the optimal alpha.

Is it better than the linear regression? If so, why might this be?

In [57]:
ridge_model = Ridge(alpha=775.2597488629465)

In [58]:
ridge_model.fit(X_Scaled,y)

Ridge(alpha=775.2597488629465)

In [60]:
ridge_scores = cross_val_score(ridge_model, X_Scaled, y, cv=10)

print (ridge_scores)
print (np.mean(ridge_scores))

[0.5822626  0.52880462 0.4869004  0.59583817 0.52396335 0.52942952
 0.52842192 0.41153448 0.39857643 0.44489752]
0.5030628996817688


Recall that the Linear Regression model had a $R^2$ score of -1.153088558418262e+28
 
The Ridge Regularized model has a much better score.

---

### 7) Find an optimal value for lasso regression alpha using `LassoCV`.

Go to the documentation and [read how LassoCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It is very similar to `RidgeCV`.

> *Hint: Again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that lasso, unlike ridge, performs best when searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide what alphas to use itself by setting the keyword argument `n_alphas=` to however many alphas you want it to search over. We recommend letting scikit-learn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if it's working, set `verbose =1`._

In [62]:
from sklearn.linear_model import Lasso

In [67]:
best_lasso = LassoCV(n_alphas=500, cv=10)

In [69]:
best_lasso = best_lasso.fit(X_Scaled,y)

In [70]:
print(best_lasso.alpha_)

0.06790539979326354


---

### 8) Cross-validate the lasso $R^2$ with the optimal alpha.

Is it better than the linear regression? Is it better than ridge? What do the differences in results imply about the issues with the data set?

In [71]:
lasso_model = Lasso(alpha=0.06790539979326354)

In [72]:
lasso_model.fit(X_Scaled,y)

Lasso(alpha=0.06790539979326354)

In [73]:
lasso_scores = cross_val_score(lasso_model, X_Scaled, y, cv=10)

print (lasso_scores)
print (np.mean(lasso_scores))

[0.61603625 0.52641159 0.53888743 0.61322432 0.54627539 0.5554072
 0.52716822 0.44849563 0.4646267  0.51838952]
0.5354922262084527


Lasso performs slightly better than Ridge

---

### 9) Look at the coefficients for variables in the lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original data set are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** If you only fit the lasso within `cross_val_score`, you'll have to refit it outside of that function to pull out the coefficients.

In [74]:
lasso_model.fit(X_Scaled,y)

Lasso(alpha=0.06790539979326354)

In [75]:
lasso_model.coef_

array([-2.82854323e-02, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -0.00000000e+00,  6.49685832e-03,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  6.22608272e-03, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  4.92571783e-03, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -1.43490539e-02, -6.86717542e-02, -3.50481654e-03,
       -0.00000000e+00,  0.00000000e+00,  7.46060170e-03, -9.00601335e-03,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00, -8.91632555e-02,  0.00000000e+00,
       -1.07292770e-02,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -

In [80]:
lasso_coef = pd.DataFrame({'Predictor Variable':X.columns, 'Coefficient':lasso_model.coef_, 'Absolute Coefficient':np.abs(lasso_model.coef_)})

lasso_coef.sort_values('Absolute Coefficient', inplace=True, ascending=False)

lasso_coef

,Predictor Variable,Coefficient,Absolute Coefficient
579,COMBINED_SHOT_TYPE:jump_shot,1.233880,1.233880
574,SHOT_TYPE:2pt_field_goal,0.866201,0.866201
566,SHOT_ZONE_BASIC:restricted_area,0.453050,0.453050
577,COMBINED_SHOT_TYPE:dunk,0.274926,0.274926
611,ACTION_TYPE:jump_shot,-0.272130,0.272130
...,...,...,...
270,SEASON_OPPONENT:mil:2009-10,-0.000000,0.000000
269,SEASON_OPPONENT:mil:2008-09,0.000000,0.000000
268,SEASON_OPPONENT:mil:2007-08,-0.000000,0.000000
267,SEASON_OPPONENT:mil:2006-07,-0.000000,0.000000


In [88]:
lasso_coef[lasso_coef['Absolute Coefficient']==0].count()

Predictor Variable      508
Coefficient             508
Absolute Coefficient    508
dtype: int64

In [94]:
X.shape

(1558, 644)

In [95]:
508/1558

0.32605905006418484

In [93]:
np.sum((lasso_model.coef_==0)==True)

508

In [90]:
print ('Percent variables zeroed out:', np.sum((lasso_model.coef_ == 0))/float(X.shape[0]))

Percent variables zeroed out: 0.32605905006418484


---

### 10) Find an optimal value for elastic net regression alpha using `ElasticNetCV`.

Go to the documentation and [read how ElasticNetCV works](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note that here you'll be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: Strength of regularization.
- `l1_ratio`: Amount of ridge vs. lasso (0 = all ridge, 1 = all lasso).
    
Do not include 0 in the search for `l1_ratio` — it won't allow it and will break.

You can use `n_alphas` for the alpha parameters instead of setting your own values, which we highly recommend.

Also, be careful setting too many l1_ratios over cross-validation folds in your search. It can take a long time if you choose too many combinations and, for the most part, there are diminishing returns in this data.

In [97]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

In [105]:
# Set up a list of alphas to check.
enet_alphas = np.linspace(0.01, 1.0, 25)

# Set up our l1 ratio. (What does this do?)
enet_ratio = enet_alphas

# Instantiate model.
enet_model = ElasticNetCV(l1_ratio=enet_ratio, n_alphas = 100, cv=10)

# Fit model using optimal alpha.
enet_model = enet_model.fit(X_Scaled, y)

# Generate predictions.
enet_model_preds = enet_model.predict(X_Scaled)
enet_model_preds_train = enet_model.predict(X_Scaled)

# Evaluate model.
print(enet_model.score(X_Scaled, y))
#print(enet_model.score(X_test, y_test))

KeyboardInterrupt: 

In [104]:
enet_model.alpha_

0.0925

---

In [102]:
l1_ratios = np.linspace(0.01, 1.0, 25)

optimal_enet = ElasticNetCV(l1_ratio=l1_ratios, n_alphas=100, cv=10,
                            verbose=1)
optimal_enet.fit(X_Scaled, y)

print (optimal_enet.alpha_)
print (optimal_enet.l1_ratio_)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

0.07271360998818006
0.9587500000000001


---

### 11) Cross-validate the elastic net $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the ridge and lasso regularized regressions?

In [106]:
enet = ElasticNet(alpha=optimal_enet.alpha_, l1_ratio=optimal_enet.l1_ratio_)

enet_scores = cross_val_score(enet, X_Scaled, y, cv=10)

print (enet_scores)
print (np.mean(enet_scores))

[0.61742795 0.52697978 0.53864379 0.61332045 0.54728018 0.55547707
 0.52548033 0.4485161  0.46407359 0.52059974]
0.5357798983416926


---

### 12. [Bonus] Compare the residuals for ridge and lasso visually.


In [50]:
# A: Maybe a jointplot?